<a href="https://colab.research.google.com/github/ronybot/NLP_HebrewEmbeding/blob/master/Data_augmentation_resnet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from keras.models import load_model
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input
import numpy as np
import requests
from io import BytesIO
from PIL import Image
from PIL import Image
import asyncio
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import csv


# Load the trained model

from google.colab import drive
import shutil
from PIL import Image





In [4]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install cairosvg


In [6]:
pip install pillow


In [7]:
pip install retrying


In [8]:
from PIL import Image
from io import BytesIO
import requests
import cairosvg
import xml.etree.ElementTree as ET
import hashlib
import aiohttp
import asyncio
from retrying import retry
import re
import base64
import time
import threading
import pickle
import io



# Global variable to be shared across threads


public_gatway = ['dweb.link','ipfs.io','cf-ipfs.com','gateway.pinata.cloud','cloudflare-ipfs.com']
SVG_TYPE = 'image/svg+xml'
GIF_TYPE = 'image/gif'
TEXT_TYPE = 'text/'
MP4_TYPE = 'video/mp4'

class image_download_manager:
    def __init__(self):
        self.base46_svg_pattern = re.compile(r'^data:image/svg+xml;base64')
        self.base46_str_svg_pattern = 'data:image/svg+xml;base64'
        self.base46_str_svg_pattern = 'data:image/svg+xml;base64'
        self.utf8_str_svg_pattern ='data:image/svg+xml;utf8'

    def _hash_url(self,url):
        # Create a new SHA-256 hash object
        sha256 = hashlib.sha256()

        # Update the hash object with the URL bytes
        sha256.update(url.encode('utf-8'))

        # Get the hexadecimal representation of the hash
        hashed_url = sha256.hexdigest()

        return hashed_url
    def _is_svg(self, content):
        """
        Check if the given content or file is in SVG format.
        """
        if isinstance(content, bytes):
            # Check if the content starts with '<?xml' (XML declaration)
            return content.startswith(b'<?xml') and b'svg' in content.lower()
        elif isinstance(content, str):
            # Check if the content starts with '<?xml' (XML declaration)
            return content.startswith('<?xml') and 'svg' in content.lower()
        else:
            return False  # Unsupported content type
    def _is_svg_content(self, content):
        try:
            root = ET.fromstring(content)
            return root.tag.lower() == "{http://www.w3.org/2000/svg}svg"
        except ET.ParseError:
            return False

    async def _fetch_data(self, url):
        # match = self.base46_svg_pattern.match(url)
        match = url.startswith(self.base46_str_svg_pattern) | url.startswith(self.utf8_str_svg_pattern)
        # encoded_data= None
        if match:
            data_index = url.find(',')
            if(data_index != -1):
                _data = url[data_index + 1:]
                img_content = _data if url.startswith(self.utf8_str_svg_pattern) else base64.b64decode(_data)
                im = Image.open(BytesIO(cairosvg.svg2png(img_content)))
                # encoded_data = match.group(1)
                # encoded_data =base64.b64decode(encoded_data, validate=True)
                return (im, 200, SVG_TYPE)

            return (None, 500, None)
        _start = time.time()
        timeout = aiohttp.ClientTimeout(total=15)
        async with aiohttp.ClientSession(timeout=timeout) as session:
            async with session.get(url) as response:
                _end = time.time()

                content = await response.read()
                if  response.content_type.startswith(TEXT_TYPE) or response.content_type == MP4_TYPE:
                    return (None, 500, response.content_type)
                im = Image.open(BytesIO(cairosvg.svg2png(file_obj=BytesIO(content)))) if response.content_type == SVG_TYPE else Image.open(BytesIO(content))
                return (im, response.status, response.content_type)

    async def _download_image_management(self, image_url):
        url=''
        if image_url.startswith('ipfs://'):
            for gatway in public_gatway:
                url = image_url.replace('ipfs://', f'https://{gatway}/ipfs/')
                try:
                    content, status, contentType = await self._fetch_data(url)
                    if status == 200:
                        return content, contentType
                except Exception as err:
                    print("An error occurred while downloading IPFS image_url " + image_url + " " +type(err).__name__  )
                await asyncio.sleep(10)


            return None, None
        try:
            _content, _status, _contentType = await self._fetch_data(image_url)
            if _status == 200:
                return _content, _contentType
            else:
                return None, None
        except Exception as err:

            return None, None

    def gif_to_images(self, gif_img: Image, frame_index=0):
    # Open the GIF file
        with gif_img as img:
            # Ensure the requested frame index is within the valid range
            frame_index = min(max(frame_index, 0), img.n_frames - 1)

            # Set the current frame to the requested frame
            img.seek(frame_index)

            # Convert the frame to RGB mode (optional, depending on your needs)
            rgb_frame = img.convert("RGB")

            # Return the RGB frame as a PIL Image
            return rgb_frame


    @retry(stop_max_attempt_number=3, wait_exponential_multiplier=1000, wait_exponential_max=60000)  # Max 1 minute between retries
    async def download_image(self,image_url, cache):
      try:
          img, contentType = await self._download_image_management(image_url)
          if img is None:
              return None
          if contentType == GIF_TYPE:
              img = self.gif_to_images(img)
          return img

      except Exception as err:

             return None




In [ ]:
# Get a list of all files in the folder
import os
folder_path = "/content/drive/My Drive/fake_nfts"

files = os.listdir(folder_path)

# Print the list of files
print("Files in the 'fake_nfts' folder:")
for file in files:
    print(file)

In [97]:

def print_image_by_path(image_path):
  image = Image.open(image_path)
  image.show()
  plt.imshow(image)
  plt.axis('off')  # Hide axis
  plt.show()


def print_image(content):

  plt.imshow(content)
  plt.axis('off')  # Turn off axis
  plt.show()

In [96]:
import csv
import os
import requests

dataset = "/content/drive/My Drive/fake_nfts/resnet50-dataset.csv"

with open(dataset, 'r', newline='') as csvfile:
            csv_reader = csv.DictReader(csvfile)
            for row in csv_reader:

                image_url = row.get('imageUrl')
                fake_image_url = row.get('fakeImageurl')
                if fake_image_url:
                  downloder = image_download_manager()
                  filename = image_url.split('/')[-1]
                  fake_filename = fake_image_url.split('/')[-1]


                  fake_image =await downloder.download_image(fake_image_url, False)
                  image = await downloder.download_image(image_url, False)

                  file      =os.path.join("/content/drive/My Drive/fake_nfts/", filename+".png")
                  fake_file =os.path.join("/content/drive/My Drive/fake_nfts/", fake_filename+".png")
                  if not os.path.exists(file):
                    print("saving " + file)
                    image.save(file,format="PNG")
                  if not os.path.exists(fake_file):
                    print("saving fake " + fake_file)
                    fake_image.save(fake_file,format="PNG")


                  #print_image(image)


In [53]:
import cv2
import numpy as np
import random




def apply_blur(image, kernel_size=(25, 25)):
    return cv2.GaussianBlur(image, kernel_size, 0)

def remove_extension(filename):
    return os.path.splitext(filename)[0]

def apply_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def apply_rotation(image):
    angle =random.uniform(-170, 170)
    print("angle" +angle)
    height, width = image.shape[:2]
    rotation_matrix = cv2.getRotationMatrix2D((width/2, height/2), angle, 1)
    return cv2.warpAffine(image, rotation_matrix, (width, height))

In [ ]:
distinct_values = set()
action = "grayscale"

with open(dataset, 'r', newline='') as csvfile:
            csv_reader = csv.DictReader(csvfile)
            for row in csv_reader:

                original_file_name= row.get('Image2_file')
                if original_file_name not in distinct_values:
                  image_path =os.path.join("/content/drive/My Drive/fake_nfts/", original_file_name)
                  distinct_values.add(original_file_name)
                  image =cv2.imread(image_path)
                  #print_image(image)
                  new_file_name =remove_extension(original_file_name)+"_"+action+".png"
                  if not os.path.exists(new_file_name):


                    new_file_path = os.path.join("/content/drive/My Drive/fake_nfts/", new_file_name)
                    transformed_image = apply_grayscale(image)
                    #print_image(transformed_image)
                    cv2.imwrite(new_file_path, transformed_image)
                    #cv2.imread('transformed_image',transformed_image)
                    print(",0.9,,9,"+original_file_name+","+new_file_name+","+action)

In [ ]:
distinct_values = set()

with open(dataset, 'r', newline='') as csvfile:
            csv_reader = csv.DictReader(csvfile)
            for row in csv_reader:

                original_file_name= row.get('Image2_file')
                if original_file_name not in distinct_values:
                  image_path =os.path.join("/content/drive/My Drive/fake_nfts/", original_file_name)
                  distinct_values.add(original_file_name)
                  image =cv2.imread(image_path)
                  new_file_name =remove_extension(original_file_name)+"_blur.png"
                  if not os.path.exists(new_file_name):


                    new_file_path = os.path.join("/content/drive/My Drive/fake_nfts/", new_file_name)
                    blurred_image = apply_blur(image)
                    cv2.imwrite(new_file_path, blurred_image)
                    image = cv2.imread(original_file_name)
                    print(",0.9,,9,"+original_file_name+","+new_file_name+",blure")



In [ ]:
pip install pymongo


In [91]:
import numpy as np
from keras.preprocessing import image
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.models import Model
from keras.layers import Dense
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

# Load the ResNet50 model with pre-trained weights
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

# Add a dense layer for classification
x = base_model.output
predictions = Dense(1, activation='sigmoid')(x)

# Create the model to be trained
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze all layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,  # Added for additional transformation
    brightness_range=[0.5, 1.5],  # Adjusting brightness
    channel_shift_range=100.0,  # Random channel shifts
    fill_mode='nearest'
)

# Specify your data directory
train_generator = train_datagen.flow_from_directory(
    directory='/content/drive/My Drive/fake_nfts',
    target_size=(224, 224),  # Same as ResNet50 input size
    batch_size=32,
    class_mode='binary'
)



94765736/94765736 [==============================] - 1s 0us/step
Found 0 images belonging to 0 classes.
